In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(Transformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # 取平均作为输出
        x = self.fc(x)
        return x

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
# # 假设输入数据的维度为(batch_size, sequence_length, input_dim)
batch_size = 16
sequence_length = 1000
input_dim = 32
num_classes = 6
d_model = 32  # Transformer模型中特征的维度
nhead = 4  # 多头自注意力头数
num_layers = 4  # Transformer编码器层数
model = Transformer(d_model, nhead, num_layers).to(device)
print(model)

Transformer(
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
    (linear1): Linear(in_features=32, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=32, bias=True)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linea

In [9]:
import onnx
import onnxruntime
from torch import nn
save_path = "C:\\Users\\a1882\Desktop\\EEG\\model_saved\\transformer_1000_100e_1.pt"
onnx_file_name = "./transformer_1000_100e__.onnx"
torch.load(save_path, map_location=torch.device('cpu'))

OrderedDict([('encoder_layer.self_attn.in_proj_weight',
              tensor([[-0.0299,  0.0612, -0.1074,  ...,  0.1874, -0.1438,  0.1018],
                      [-0.2022,  0.1797, -0.0896,  ...,  0.0619,  0.1461, -0.0290],
                      [ 0.1697,  0.1872, -0.1305,  ...,  0.0373, -0.1238,  0.0025],
                      ...,
                      [ 0.0434, -0.0207, -0.1276,  ..., -0.2075, -0.1074,  0.0820],
                      [-0.0797,  0.0602, -0.1537,  ..., -0.1581, -0.0481, -0.1953],
                      [-0.1915,  0.0174,  0.1310,  ...,  0.0938, -0.1755, -0.0699]])),
             ('encoder_layer.self_attn.in_proj_bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [13]:
# model.eval()
# batch_size = 1
# dummy_input = torch.randn(batch_size, 1000,32 ,requires_grad=True ,dtype=torch.float32)
# output = model(dummy_input)
# torch.onnx.export(model, dummy_input, onnx_file_name, verbose=True, input_names=['input'], output_names=['output'],
#  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},opset_version=12)

In [24]:
signal = pd.read_csv("C:\\Users\\a1882\Desktop\EEG\eegdata\slice_data\\rest_zyy_06_epocflex_2023.03.22t16.46.00+08.00.md.bp.csv_14.csv").T
input = torch.from_numpy(signal.values)
print(input.shape)
input = input.unsqueeze(0)
with torch.no_grad():
    output = model(input.float())
    print(output)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    print(probabilities)

    _, pred = torch.max(output, dim=1)  # 找到预测分数最大的类别，得到预测类别

    label_map = {0: 'lefthand', 1:'read' ,  2:'rest', 3: 'walkbase', 4: 'walkl' ,5: 'walkfocus'}
    print(label_map[pred.item()])

torch.Size([1000, 32])
tensor([[ 0.0413, -0.2721,  0.3691,  0.3503,  0.4697,  0.4085]])
tensor([[0.1341, 0.0980, 0.1861, 0.1826, 0.2058, 0.1935]])
walkl
